# Spaceship KNN Submission

Hello there whoever handles this aku lupa, ini cuma coba-coba aja si pake KNeighborsClassifier buat test submission. Feel free to modify ya, lov u makasi semangat. -Egar

In [ ]:
!pip install catboost==1.2.5


[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_auc_score
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA
from catboost import CatBoostClassifier
from sklearn.preprocessing import MinMaxScaler

# Bring the Data In

In [ ]:
# Load data
X_df = pd.read_csv('preprocessedbankchurn_train.csv')
X_df.drop("Exited", axis=1, inplace=True)
y_df = pd.read_csv('preprocessedbankchurn_train.csv')['Exited']
X_submission = pd.read_csv('preprocessedbankchurn_test.csv')

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, test_size=0.2, random_state=42)

# Model Training

In [ ]:
# Membuat pipeline
pipeline = Pipeline([
    ('scaler', MinMaxScaler()), 
    ('classifier', CatBoostClassifier(random_seed=42, verbose=0))
])


param_grid = {
    'classifier__iterations': [1000],
    'classifier__learning_rate': [0.01],
    'classifier__depth': [10],
    'classifier__l2_leaf_reg': [10],
    'classifier__border_count': [128],
    'classifier__loss_function': ['CrossEntropy']
}


# Menggunakan StratifiedKFold untuk handling imbalanced data
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

In [ ]:
# GridSearchCV setup
grid_search = GridSearchCV(pipeline, param_grid, cv=cv, scoring='roc_auc', verbose=1)
grid_search.fit(X_train, y_train)

# Evaluate the model
best_model = grid_search.best_estimator_
y_pred_proba = best_model.predict_proba(X_test)[:, 1]  # Get probabilities for the positive class
print("Best parameters:", grid_search.best_params_)
print("ROC-AUC on test set:", roc_auc_score(y_test, y_pred_proba))

Fitting 10 folds for each of 1 candidates, totalling 10 fits
Best parameters: {'classifier__border_count': 128, 'classifier__depth': 10, 'classifier__iterations': 1000, 'classifier__l2_leaf_reg': 10, 'classifier__learning_rate': 0.01, 'classifier__loss_function': 'CrossEntropy'}
ROC-AUC on test set: 0.901263115531918


In [ ]:
# Evaluation on the test set
y_pred = best_model.predict(X_test)
print(classification_report(y_test, y_pred))
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

# Fit the model on all training data
grid_search.fit(X_df, y_df)

              precision    recall  f1-score   support

           0       0.89      0.95      0.92     26052
           1       0.76      0.58      0.65      6955

    accuracy                           0.87     33007
   macro avg       0.82      0.76      0.79     33007
weighted avg       0.86      0.87      0.86     33007

Confusion Matrix:
[[24751  1301]
 [ 2941  4014]]
Fitting 10 folds for each of 1 candidates, totalling 10 fits


GridSearchCV(cv=StratifiedKFold(n_splits=10, random_state=42, shuffle=True),
             estimator=Pipeline(steps=[('scaler', MinMaxScaler()),
                                       ('classifier',
                                        <catboost.core.CatBoostClassifier object at 0x7f52e5c3b490>)]),
             param_grid={'classifier__border_count': [128],
                         'classifier__depth': [10],
                         'classifier__iterations': [1000],
                         'classifier__l2_leaf_reg': [10],
                         'classifier__learning_rate': [0.01],
                         'classifier__loss_function': ['CrossEntropy']},
             scoring='roc_auc', verbose=1)

In [ ]:
# Melakukan cross-validation dengan GridSearchCV
scores = cross_val_score(grid_search, X_df, y_df, cv=5)

# Menampilkan hasil cross-validation
print("Cross-validation scores:", scores)
print("Mean cross-validation score:", scores.mean())

Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Fitting 10 folds for each of 1 candidates, totalling 10 fits
Cross-validation scores: [0.90313274 0.89761761 0.90010359 0.89999802 0.89767529]
Mean cross-validation score: 0.8997054473684797


In [ ]:
# Evaluation on the test set
y_pred = grid_search.predict(X_test)
print(classification_report(y_test, y_pred))
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

# Fit the model on all training data
grid_search.fit(X_df, y_df)

              precision    recall  f1-score   support

           0       0.90      0.96      0.93     26052
           1       0.78      0.60      0.68      6955

    accuracy                           0.88     33007
   macro avg       0.84      0.78      0.80     33007
weighted avg       0.88      0.88      0.87     33007

Confusion Matrix:
[[24908  1144]
 [ 2788  4167]]
Fitting 10 folds for each of 1 candidates, totalling 10 fits


GridSearchCV(cv=StratifiedKFold(n_splits=10, random_state=42, shuffle=True),
             estimator=Pipeline(steps=[('scaler', MinMaxScaler()),
                                       ('classifier',
                                        <catboost.core.CatBoostClassifier object at 0x7f52e5c3b490>)]),
             param_grid={'classifier__border_count': [128],
                         'classifier__depth': [10],
                         'classifier__iterations': [1000],
                         'classifier__l2_leaf_reg': [10],
                         'classifier__learning_rate': [0.01],
                         'classifier__loss_function': ['CrossEntropy']},
             scoring='roc_auc', verbose=1)

In [ ]:
# Predict probabilities for submission data
y_submission_proba = grid_search.predict_proba(X_submission)[:, 1]  # Probabilities of Exited
print(y_submission_proba)

# Prepare submission dictionary
submission_dict = {'id': X_submission['id'], 'Exited': y_submission_proba}
submission_dict = pd.DataFrame(submission_dict)
submission_dict

[0.01605637 0.85669724 0.0191635  ... 0.01918493 0.18429122 0.15059033]


,id,Exited
0,165034,0.016056
1,165035,0.856697
2,165036,0.019163
3,165037,0.186274
4,165038,0.369112
...,...,...
110018,275052,0.031479
110019,275053,0.111149
110020,275054,0.019185
110021,275055,0.184291


In [ ]:
submission_dict.to_csv('bankChurn_cat5.csv', index=False)

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=2a79941c-6614-47fe-9427-0e9f23998893' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>